In [1]:
import json

def read_json(file):
    with open(file) as f:
        data = json.load(f)
    return data

data = read_json("data/v1_1_train_nus_ext_sample.json")
results = read_json("outputs/v1_1_train_nus_ext_sample_llama_results.json")

In [15]:
from termcolor import colored

def display_next_prediction(data, results):
    results_map = {}

    for result in results:
        results_map[result["id"]] = result

    for scene_id, scene in data.items():
        frames = scene["key_frames"]
        
        print(f"{colored('Scene:', 'blue')} {scene_id}")

        for frame_id, frame in frames.items():
            qa = frame["QA"]
            images = frame["image_paths"]
            tasks = ["perception", "prediction", "planning", "behavior"]
            
            print(f"{colored('Frame:', 'blue')} {frame_id}")

            for i, task in enumerate(tasks):
                for j, question in enumerate(qa[task]):
                    offset = sum([len(qa[t]) for t in tasks[:i]])
                    question_id = f"{scene_id}_{frame_id}_{j+offset}"
                    result = results_map[question_id]
                    print(f"{colored('Question:', 'blue')} {question['Q']}")
                    print(f"{colored('Answer:', 'blue')} {question['A']}")
                    print(f"{colored('Model Input:', 'blue')} {result['question']}")
                    print(f"{colored('Model Output:', 'blue')} {result['answer']}")
                    print(f"{colored('Images:', 'blue')} {images}")
                    print()
                    yield
            
            print()

In [16]:
predictions = iter(display_next_prediction(data, results))

In [17]:
next(predictions)

Scene: 1fa63d31aaa7459383b4414715cd12db
Frame: 30de1c4c6e1f41e59261a5c5e8ca38b0
Question: What are the important objects in the current scene? Those objects will be considered for the future reasoning and driving decision.
Answer: There is a white truck to the front of the ego vehicle, a black car to the front of the ego vehicle, and a white sedan to the back of the ego vehicle. The IDs of these objects are <c1,CAM_FRONT,486.7,520.0>, <c2,CAM_FRONT,1235.8,584.2>, and <c3,CAM_BACK,1180.0,630.8>.
Model Input: <image>
What are the important objects in the current scene? Those objects will be considered for the future reasoning and driving decision.
Model Output: There is a white sedan to the front of the ego vehicle, a white sedan to the front of the ego vehicle, a white sedan to the back of the ego vehicle, and a white sedan to the front of the ego vehicle. The IDs of these objects are <c1,CAM_FRONT,1000.0,500.0>, <c2,CAM_BACK,1000.0,500.0>, <c3,CAM_FRONT,1000.0,500.0>, and <c4,CAM_BACK,